<a href="https://colab.research.google.com/github/DongUk-Park/RnD/blob/main/POI/NAIS/MF_BPR_POI_distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Config

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import random
import csv
import copy

import os
import time
import argparse
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.backends.cudnn as cudnn

# Data Utils

### load_data

In [18]:
def load_data():
  train_dataset_path = '/content/history_train.csv'
  validation_dataset_path = '/content/history_validation.csv'
  test_dataset_path = '/content/history_test.csv'

  train_history_data = []
  validation_history_data = []
  test_history_data = []

  with open(train_dataset_path, 'r', newline='') as csv_train:
      csv_reader = csv.reader(csv_train)
      header = next(csv_reader)  # 헤더 행, 실제론 data 바로 들어옴
      header = [int(item) for item in header]
      train_history_data.append(header)
      for row in csv_reader:
          r = [int(item) for item in row]
          train_history_data.append(r)

  with open(validation_dataset_path, 'r', newline='') as csv_validation:
      csv_reader = csv.reader(csv_validation)
      header = next(csv_reader)  # 헤더 행, 실제론 data 바로 들어옴
      header = [int(item) for item in header]
      validation_history_data.append(header)
      for row in csv_reader:
          r = [int(item) for item in row]
          validation_history_data.append(r)

  with open(test_dataset_path, 'r', newline='') as csv_test:
      csv_reader = csv.reader(csv_test)
      header = next(csv_reader)  # 헤더 행, 실제론 data 바로 들어옴
      header = [int(item) for item in header]
      test_history_data.append(header)
      for row in csv_reader:
          r = [int(item) for item in row]
          test_history_data.append(r)

  num_users = len(train_history_data) # 필라델피아의 사용자 수 : 15359명
  num_items = business_id_info.shape[0] # 필라델피아의 가게 수 : 14586개, 0 ~ 14585

  negative_train_history_data = []
  negative_validation_history_data = []
  negative_test_history_data = []

  with open('/content/history_train_negative.csv', 'r', newline='') as ng_train, open('/content/history_test_negative.csv', 'r', newline='') as ng_test:
      ng_train_reader = csv.reader(ng_train)
      ng_test_reader = csv.reader(ng_test)
      for row in ng_train_reader:
        r = [int(item) for item in row]
        negative_train_history_data.append(r)
      for row in ng_test_reader:
        r =  [int(item) for item in row]
        negative_test_history_data.append(r)

  target_idx_list = copy.deepcopy(test_history_data)
  for i in range(num_users):
    target_idx_list[i].extend(negative_test_history_data[i])

  business_id_info = pd.read_csv('/content/business_info_in_philadelphia.csv', names = ['business_id', 'latitude', 'longitude', 'city'])
  item_list = business_id_info['business_id'].tolist()
  latitude_list = business_id_info['latitude'].tolist()
  longitude_list = business_id_info['longitude'].tolist()


  return item_list, latitude_list, longitude_list, num_users, num_items, train_history_data, validation_history_data, test_history_data, negative_train_history_data, negative_test_history_data, target_idx_list

### load_all

In [11]:
def load_all(train_history_data, test_history, user_num, item_num):
	""" We load all the three file here to save time in each epoch. """
	print(f"load all 속 {user_num}, {item_num}")

	train_data = []
	for user_idx, visited_list in enumerate(train_history_data):
		for b_id in visited_list:
			train_data.append([user_idx, b_id])

	# load ratings as a dok matrix
	# ratings를 dok(Dictionary of Keys, 좌표:key, 원소 값: value) 형식의 sparse matrix로 변환
	train_mat = sp.dok_matrix((user_num, item_num), dtype=np.float32)
	for x in train_data:
		train_mat[x[0], x[1]] = 1.0

	# test_rating 파일을 읽어 test_data로 로드
	test_data = []
	for user_idx, visited_list in enumerate(test_history):
		for b_id in visited_list:
			test_data.append([user_idx, b_id])

	return train_data, test_data, train_mat

## Distance_prob

In [6]:
# Haversine 거리 계산 함수
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # 지구 반지름 (km)
    return c * r

def distance_prob_check(x):
  a, b = 0.02059073981980995 , -2.4172468185424805
  if x == 0.0:
    return 0
  y = a * (x**b)

  if y>1:
    y = 1
  return y

In [ ]:
def distance_prob(target_idx_list, lat_list, long_list):
  distance_prob_list = []
  for user, visit_list in enumerate(target_idx_list):
    user_prob_list = []
    for b_id in visit_list:
      min_dist = 9999
      lat1 = lat_list[b_id]
      long1 = long_list[b_id]
      for comp_b_id in visit_list:
        if b_id == comp_b_id:
          continue
        lat2 = lat_list[comp_b_id]
        long2 = long_list[comp_b_id]
        dist = haversine(lat1, long1, lat2, long2)
        if min_dist > dist:
          min_dist = dist
      prob = distance_prob_check(min_dist)
      user_prob_list.append(prob)
  distance_prob_list.append(user_prob_list)
  return distance_prob_list

[1, 0.5062683477350807, 1, 0.004364020392940585, 1, 1, 1, 0.6434971353677158, 1, 0.03649965115955932, 0.25666259965085203, 0.5932454964259025, 1, 0.3973781762991897, 0.028821709346761943, 0.026426402202928263, 1, 0.037712817533305854, 0.2493088265350592, 1, 1, 1, 0.41772708235931927, 1, 0.06682949966145417, 1, 0.024737515091624602, 1, 1, 1, 0.013533013561494996, 0.5150062031186471, 0.011953154978114569, 1, 0.08271545887236129, 1, 0.1964851353020116, 0.008625643283514348, 0.10217434901306069, 0.06538186218425203, 0.049018794151038866, 1, 1, 0.2575239276251131, 1, 0.2575239276251131, 0.558134266998659, 0.10432556243219912, 0.05605453152935245, 0.4215041237990345, 1, 0.6145118480268044, 1, 1, 1, 0.26009510078500403, 1, 1, 0.5589525772049392, 0.5096942803782982, 1, 0.08159350457236914, 1, 0.1619210042501226, 0.563066553260938, 0.014119771373554895, 1, 1, 0.013626608381887458, 0.22617371603495623, 1, 0.31214135989589914, 1, 0.2075139540148934, 0.7593254273731245, 1, 1, 0.0832121807179334, 0

### BPRData

In [7]:
# 데이터셋 클래스 정의
class BPRData(data.Dataset):
	def __init__(self, features,
				num_item, train_mat=None, num_ng=0, is_training=None, ng_dataset = None):
		super(BPRData, self).__init__()
		"""
			Note that the labels are only useful when training, we thus
			add them in the ng_sample() function.
		"""
		self.features = features #training, test dataset
		self.num_item = num_item
		self.train_mat = train_mat
		self.num_ng = num_ng
		self.is_training = is_training
		self.ng_dataset = ng_dataset

	def ng_sample(self):
		assert self.is_training, 'no need to sampling when testing'
		#훈련 데이터셋에 대하여 Negativesample 만듦
		self.features_fill = []
		for x in self.features: # training_dataset에 있는 positive data에 랜덤한 negative data를 생성해 매치해줌
			u, i = x[0], x[1]

			for t in range(self.num_ng):
				j = np.random.randint(0, self.num_item) # num_items == len(item_list), item_list에서 Negative sample을 골라야하기 때문에 인덱스를 골라줌
				ng_data = self.ng_dataset[u]
				sampled_indices = random.sample(range(len(ng_data)-1), self.num_ng)
				for idx in sampled_indices:
					j = int(ng_data[idx])
					self.features_fill.append([u, i, j])


	def __len__(self):
		return self.num_ng * len(self.features) if \
				self.is_training else len(self.features)

	def __getitem__(self, idx):
		features = self.features_fill if \
				self.is_training else self.features

		user = features[idx][0]
		item_i = features[idx][1]
		item_j = features[idx][2] if \
				self.is_training else features[idx][1]

		return user, item_i, item_j


# MFbpr

In [8]:
class BPR(nn.Module):
	def __init__(self, user_num, item_num, factor_num):
		super(BPR, self).__init__()
		"""
		user_num: 사용자 수
		item_num: 아이템 수
		factor_num: 예측할 factors의 수
		"""
		 # 사용자와 아이템을 factor 수 만큼 임베딩하는 레이어를 생성
		self.embed_user = nn.Embedding(user_num, factor_num)
		self.embed_item = nn.Embedding(item_num, factor_num)

		# 임베딩 레이어의 가중치를 정규 분포에서 초기화한다.
		nn.init.normal_(self.embed_user.weight, std=0.01)
		nn.init.normal_(self.embed_item.weight, std=0.01)

	def forward(self, user, item_i, item_j):
		"""
        Parameters:
            user (torch.Tensor): 사용자 ID list
            item_i (torch.Tensor): positive 아이템 list
            item_j (torch.Tensor): negative 아이템 list
        Returns:
            torch.Tensor: 긍정적인 아이템에 대한 예측 값
            torch.Tensor: 부정적인 아이템에 대한 예측 값
		"""
		user = self.embed_user(user)
		item_i = self.embed_item(item_i)
		item_j = self.embed_item(item_j)

		prediction_i = (user * item_i).sum(dim=-1)  # user * i matrix 생성
		prediction_j = (user * item_j).sum(dim=-1) # user * j matrix 생성

		return prediction_i, prediction_j

#Evaluate

In [34]:
def eval(model, num_users, testPositives, target_idx_list, target_idx_dist_prob_list):
	model.eval()
	hit = [0,0,0,0,0]
	precision = [0,0,0,0,0]
	recall = [0,0,0,0,0]

	k=[10,20,30,40,50]

	with torch.no_grad():
		for u in range(num_users):
			target_idx = target_idx_list[u]
			user_list = [u] * len(target_idx)
			user_tensor = torch.tensor(user_list)
			target_tensor = torch.tensor(target_idx)
			dist_tensor = torch.tensor(target_idx_dist_prob_list[u])

			prediction_i, prediction_j = model(user_tensor, target_tensor, target_tensor)

			print(f"prediction_i : {prediction_i.shape}")
      #각각의 유저에 대해서 target_idx에 있는 가게 거리 정보를 더해줘야함


			#print(f"dist_prob : {dist_tensor}")
			alpha = 0.7
			prediction_i = alpha * prediction_i + (1 - alpha) * dist_tensor

			for i in range(len(k)):
				_, indices = torch.topk(prediction_i, k[i])
				recommends = [target_idx[i] for i in indices]
				topSet = set(recommends)

				positives = set(testPositives[u])
				count = len(topSet & positives)

				precision[i] += (count/k[i]) / num_users
				recall[i] += (count/len(testPositives[u]))/ num_users
				if count >0:
					hit[i] += 1/ num_users

	return recall , precision, hit

#Main

In [36]:
"""#Main"""
class Args:
	def __init__(self):
		self.lr = 0.01 # learning rate
		self.lamda = 0.001 # model regularization rate
		self.batch_size = 4096 # batch size for training
		self.epochs = 25 # training epoches
		self.top_k = 10 # compute metrics@top_k
		self.factor_num = 32 # predictive factors numbers in the modelㄹ
		self.num_ng = 4 # sample negative items for training
		self.test_num_ng = 99 # sample part of negative items for testing
		self.out = True # save model or not
		#self.gpu = "0" # gpu card ID


if __name__ == "__main__":
	args = Args()

	item_list, latitude_list, longitude_list, num_users, num_items, train_history_data, validation_history_data, test_history_data, negative_train_history_data, \
	negative_test_history_data, target_idx_list_for_test = load_data()

	############################## PREPARE DATASET ##########################
	train_data, test_data, train_mat = load_all(train_history_data, test_history_data, num_users, num_items)
	target_idx_dist_prob_list = distance_prob(target_idx_list_for_test, latitude_list, longitude_list)


	# construct the train and test datasets
	# 훈련 및 테스트 데이터셋 생성
	train_dataset = BPRData(
			train_data, num_items, train_mat, args.num_ng, True, negative_train_history_data)

	test_dataset = BPRData(
			test_data, num_items, train_mat, 0, False)

	# DataLoader로 배치 처리를 위한 데이터 로더 생성
	train_loader = data.DataLoader(train_dataset,
			batch_size=args.batch_size, shuffle=True, num_workers=0)  # num_workers를 0으로 변경하여 CPU에서 처리, 기존 코드는 GPU를 사용
	test_loader = data.DataLoader(test_dataset,
			batch_size=args.test_num_ng+1, shuffle=False, num_workers=0)

	########################### CREATE MODEL #################################
	# BPR 모델 생성
	model = BPR(num_users, num_items, args.factor_num)

	# 옵티마이저 생성 (SGD 사용)
	optimizer = optim.SGD(
				model.parameters(), lr=args.lr, weight_decay=args.lamda)
	# writer = SummaryWriter() # for visualization

	########################### TRAINING #####################################
	count, best_hr = 0, 0

	avg_recall = [0, 0, 0, 0, 0]
	avg_precision = [0, 0, 0, 0, 0]
	avg_hit = [0, 0, 0, 0, 0]
	for epoch in range(args.epochs):
		model.train()  # 모델을 학습 모드로 설정
		start_time = time.time() # 시작시간 기록
		num = 0
		train_loader.dataset.ng_sample() # negative 예제 샘플링
		for user, item_i, item_j in train_loader:
			model.zero_grad() # 그래디언트 초기화
			prediction_i, prediction_j = model(user, item_i, item_j)
			loss = - (prediction_i - prediction_j).sigmoid().log().sum() # BPR 손실 함수 계산
			loss.backward() # 역전파 및 그래디언트 계산
			optimizer.step() # 옵티마이저 업데이트
			# writer.add_scalar('data/loss', loss.item(), count)
			count += 1

		model.eval() # 모델을 평가 모드로 설정
		recall , precision, hit = eval(model, num_users, test_history_data, target_idx_list_for_test, target_idx_dist_prob_list)

		avg_recall = [x + y for x, y in zip(avg_recall, recall)]
		avg_precision = [x + y for x, y in zip(avg_precision, precision)]
		avg_hit = [x + y for x, y in zip(avg_hit, hit)]

		elapsed_time = time.time() - start_time  # 소요시간 계산

		print("@" * 30)
		print("#" * 30)
		print(f"Epoch : {epoch}, Used_Time : {elapsed_time}")
		print(f"hit : {hit}")
		print(f"precision : {precision}")
		print(f"recall : {recall}")

	for i in range(5):
		print("들어왔다")
		avg_hit[i] /= args.epochs
		avg_recall[i] /= args.epochs
		avg_precision[i] /= args.epochs
	print("@" * 30)
	print("#" * 30)
	print("final")
	print(f"hit : {avg_hit}")
	print(f"precision : {avg_precision}")
	print(f"recall : {avg_recall}")

15360 14586
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])


RuntimeError: ignored